In [1]:
import os
import tifffile as tiff
import numpy as np
from PIL import Image
import cv2
import shutil
import random
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
import random
import warnings

warnings.filterwarnings("ignore")
random.seed(241)

os.environ["CUDA_VISIBLE_DEVICES"] = "0" # Set to GPU 0 on Training
os.environ["WORLD_SIZE"] = "1"

In [14]:
model = "yolov5m-seg.yaml" # Architecture Recepie
data_name = "20ch_minmax" # Dataset Name
fold = 5 #CV Fold


### Preprocess Image

In [4]:
def get_file_names(path, remove_extension=False) :
    if remove_extension :
        return sorted([i.split('.')[0] for i in os.listdir(path) if i[0] != '.'])
    else :
        return sorted([i for i in os.listdir(path) if i[0] != '.'])

In [5]:
def read_images(path) :
    file_names = get_file_names(path)

    images = []
    for file_name in file_names :
        file_path = os.path.join(path, file_name)
        
        image = tiff.imread(file_path)

        images.append(np.array(image, dtype=np.float32))
    
    return images

In [6]:
def select_band(band, path, preprocess=None) :
    images = read_images(path)

    extracted_channel = []

    for image in images :
        extracted_channel.append(image[:, :, band])

    if preprocess == 'min_max' :
        min, max = find_min_max(extracted_channel)
        extracted_channel = norm_min_max(extracted_channel, min, max)

    return extracted_channel

In [7]:
def add_bands(a, b):
    result = []
    
    for img_a, img_b in zip(a, b):
        result.append(img_a + img_b)
    
    return result

def subtract_bands(a, b) :
    result = []
    
    for img_a, img_b in zip(a, b):
        result.append(img_a - img_b)

    return result

def multiply_bands(a, const) :
    result = []

    for img in a :
        result.append(img * const)

    return result

def divide_bands(a, const) :
    result = []

    for img in a :
        result.append(img / const)

    return result

In [8]:
def find_min_max(arr) :
    min_val = np.min(arr[0])
    max_val = np.max(arr[0])

    for img in arr:
        min_val = np.minimum(min_val, np.min(img))
        max_val = np.maximum(max_val, np.max(img))

    return min_val, max_val

In [9]:
def rescale(arr, min=0, max=1, ch=None) :
    rescaled_img = []

    for img in arr :
        min, max = np.nanmin(img), np.nanmax(img)
        img = (img - min)/(max-min)
        rescaled_img.append(img)

    return rescaled_img

In [10]:
def formula(bandA, bandB) :
    new_band = []
    for i in range(len(bandA)) :
        a, b = bandA[i], bandB[i]
        processed_band = (a-b) / ((a+b) + 1e-10)
        
        new_band.append(processed_band)

    return new_band

In [11]:
def export_images(images, source_path, export_path) :
    file_names = get_file_names(source_path)


    for i in range(len(file_names)) :
        image = images[i]
        H, W, C = image.shape
        
        image = cv2.resize(image, (W * 10, H * 10), interpolation=cv2.INTER_LINEAR)
        
        file_path = os.path.join(export_path, file_names[i])
        tiff.imwrite(file_path, image)

def export_images_jpg(channels, source_path, export_path):
    file_names = get_file_names(source_path)
    
    for i in range(len(file_names)):
        # Stack the channels to form an RGB image
        rgb_image = np.stack((channels[0][i], channels[1][i], channels[2][i]), axis=-1)

        # Create an Image object
        pil_image = Image.fromarray(rgb_image.astype(np.uint8), 'RGB')
        # Define the file path
        file_path = os.path.join(export_path, file_names[i].replace('tif', 'jpg'))
        # Save the image as JPEG
        pil_image.save(file_path, 'JPEG')

In [12]:
def plot_heatmap(tensor) :
    tensor_array = tensor
    
    # Plot heatmap
    plt.figure(figsize=(8, 6))
    sns.heatmap(tensor_array, cmap='viridis', annot=False, xticklabels=False, yticklabels=False, linewidths=0)
    plt.title('Heatmap')
    plt.xlabel('X-axis')
    plt.ylabel('Y-axis')
    plt.show()

In [13]:
def export_labels(label_source_path, label_export_path) :
    file_names = get_file_names(label_source_path)

    for file_name in file_names :
        file_path = os.path.join(label_source_path, file_name)
        mask = tiff.imread(file_path)
        H, W = mask.shape
        
        mask = cv2.resize(mask, (W * 10, H * 10), interpolation=cv2.INTER_LINEAR)
        _, mask = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY)

        H, W = mask.shape
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
        # convert the contours to polygons
        polygons = []
        for cnt in contours:
            if cv2.contourArea(cnt) > 0:
                polygon = []
                for point in cnt:
                    x, y = point[0]
                    polygon.append(x / W)
                    polygon.append(y / H)
                polygons.append(polygon)
    
        # print the polygons
        file_name = file_name.replace('mask', 's2_image')[:-4]+'.txt'
        with open(os.path.join(label_export_path, file_name), 'w') as f:
            for polygon in polygons:
                for p_, p in enumerate(polygon):
                    if p_ == len(polygon) - 1:
                        f.write('{}\n'.format(p))
                    elif p_ == 0:
                        f.write('0 {} '.format(p))
                    else:
                        f.write('{} '.format(p))
    
            f.close()

In [15]:
def make_data_directory(data_name, overwrite=False) :
    working_dir = os.getcwd()
    container_dir = os.path.join(working_dir, 'preprocessed_data')
    data_dir = os.path.join(container_dir, data_name)
    images_dir = os.path.join(data_dir, 'images')
    labels_dir = os.path.join(data_dir, 'labels')

    os.makedirs(container_dir, exist_ok=True)

    try :
        os.makedirs(data_dir)
        
    except (FileExistsError) :
        
        if overwrite :
            print('Data name exists, your data will be overwritten')

        else :
            print('Data name exists, quitting...')
            return

    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

In [16]:
image_path = 'dataset/train/s2_image/'
label_path = 'dataset/train/mask/'

image_source_path = os.path.join(os.getcwd(), image_path)
image_export_path = os.path.join(os.getcwd(), f'preprocessed_data/{data_name}/images/')
label_source_path = os.path.join(os.getcwd(), label_path)
label_export_path = os.path.join(os.getcwd(), f'preprocessed_data/{data_name}/labels/')

make_data_directory(data_name, overwrite=True)

print('Source Path : ', image_source_path)
print('Export Path : ', image_export_path)
print('Images Count : ', len(get_file_names(image_source_path)))

ch_num = [0,1,2,3,4,5,6,7,8,9,10,11]
channels = []
for i in ch_num :
    ch = select_band(i, image_source_path)
    channels.append(ch)

for i in range(1, 4) :
    for j in range(1, 4) :
        if i != j :
            ch = formula(channels[i], channels[j])
            channels.append(ch)

for i in range(10, 12) :
    for j in range(10, 12) :
        if i != j :
            ch = formula(channels[i], channels[j])
            channels.append(ch)
    
images = [np.stack([channels[i][j] for i in range(len(channels))], axis=-1, dtype=np.float32) for j in range(len(channels[0]))]
preprocessed_images = []
for image in images :
    min_val, max_val = np.min(image), np.max(image)
    image = (image - min_val) / (max_val - min_val) * 255
    preprocessed_images.append(image)
    
print('Exporting...')


export_images(preprocessed_images, image_source_path, image_export_path)
export_labels(label_source_path, label_export_path)

print('Done!!')

Source Path :  /datadisk2/c241_ml02/workspace/train_github/dataset/train/s2_image/
Export Path :  /datadisk2/c241_ml02/workspace/train_github/preprocessed_data/20ch_minmax/images/
Images Count :  2066
Exporting...
Done!!


### Create 5 Fold Cross Validation

In [17]:
def create_fold(file_names, fold) :
    random.shuffle(file_names)
    
    fold_sizes = [len(file_names) // fold for i in range(fold)]

    for i in range(len(file_names) - sum(fold_sizes)) :
        fold_sizes[-1] += 1

    fold_data = []
    for i in range(len(fold_sizes)) :
        data = file_names[sum(fold_sizes[:i]): sum(fold_sizes[:i+1])]
        fold_data.append(data)

    return fold_data

In [18]:
def make_train_directory(data_name, fold) :
    working_dir = os.getcwd()
    container_path = os.path.join(working_dir, 'train_data')
    data_path = os.path.join(container_path, data_name)
    config_path = os.path.join(data_path, 'configs')

    os.makedirs(container_path, exist_ok=True)
    os.makedirs(data_path, exist_ok=True)
    os.makedirs(config_path, exist_ok=True)
    
    for i in range(fold) :
        fold_path = os.path.join(data_path, f'fold_{(i+1)}')
        train_path = os.path.join(fold_path, 'train')
        val_path = os.path.join(fold_path, 'val')
        
        os.makedirs(fold_path, exist_ok=True)
        os.makedirs(train_path, exist_ok=True)
        os.makedirs(val_path, exist_ok=True)

        for path in [train_path, val_path] :  
            images_dir = os.path.join(path, 'images')
            labels_dir = os.path.join(path, 'labels')

            os.makedirs(images_dir, exist_ok=True)
            os.makedirs(labels_dir, exist_ok=True)

In [19]:
def copy_data(data_name, source_path, destination_path, part) :

    for file_name in data_name :
        image_file_name = file_name + '.tif'
        label_file_name = file_name + '.txt'

        image_source_path = os.path.join(source_path, 'images', image_file_name)
        image_destination_path = os.path.join(destination_path, part, 'images', image_file_name)

        label_source_path = os.path.join(source_path, 'labels', label_file_name)
        label_destination_path = os.path.join(destination_path, part, 'labels', label_file_name)

        shutil.copy(image_source_path, image_destination_path)
        shutil.copy(label_source_path, label_destination_path)

In [20]:
def make_fold(data_name, fold_data, source_path, destination_path) :
    
    for i in range(len(fold_data)) :
        train_data = []
        fold_destination_path = os.path.join(destination_path, f'fold_{(i+1)}')
        
        for fold in fold_data :
            if fold_data[i] == fold :
                copy_data(fold_data[i], source_path, fold_destination_path, 'val')
            else :
                train_data += fold

        copy_data(train_data, source_path, fold_destination_path, 'train')
            

In [21]:
def write_configs(data_name, fold, data_path, config_path) :    
    for i in range(fold) :
        fold_path = os.path.join(data_path, f'fold_{(i+1)}')
        with open(os.path.join(config_path, f'fold_{(i+1)}.yaml'), 'w') as yaml :
            config = f"""path: {fold_path}
train: train
val: val

nc: 1
names: ['solarpanel']"""

            yaml.write(config)

In [22]:
def check_image_count(destination_path, fold) :
    for i in range(fold) :
        fold_path = os.path.join(destination_path, f'fold_{(i+1)}')
        train_image_path = os.path.join(fold_path, 'train', 'images')
        train_label_path = os.path.join(fold_path, 'train', 'labels')
        val_image_path = os.path.join(fold_path, 'val', 'images')
        val_label_path = os.path.join(fold_path, 'val', 'labels')
        
        print(f'=== Fold {(i+1)} ===')
        print('Train Images:', len(os.listdir(train_image_path)))
        print('Train Labels :', len(os.listdir(train_label_path)))
        print('Val Images :', len(os.listdir(val_image_path)))
        print('Val Labels :', len(os.listdir(val_label_path)))

In [23]:
working_dir = os.getcwd()
source_path = os.path.join(working_dir, 'preprocessed_data', data_name)
destination_path = os.path.join(working_dir, 'train_data', data_name)
config_path = os.path.join(destination_path, 'configs')

file_names = get_file_names(os.path.join(source_path, 'images'), remove_extension=True)
make_train_directory(data_name, fold)

fold_data = create_fold(file_names, fold)
make_fold(data_name, fold_data, source_path, destination_path)
write_configs(data_name, fold, destination_path, config_path)

In [24]:
print('\n=== Image Count per Fold ===\n')
check_image_count(destination_path, fold)


=== Image Count per Fold ===

=== Fold 1 ===
Train Images: 1653
Train Labels : 1653
Val Images : 413
Val Labels : 413
=== Fold 2 ===
Train Images: 1653
Train Labels : 1653
Val Images : 413
Val Labels : 413
=== Fold 3 ===
Train Images: 1653
Train Labels : 1653
Val Images : 413
Val Labels : 413
=== Fold 4 ===
Train Images: 1653
Train Labels : 1653
Val Images : 413
Val Labels : 413
=== Fold 5 ===
Train Images: 1652
Train Labels : 1652
Val Images : 414
Val Labels : 414


### Train YOLOv5

In [ ]:
!python yolov5/segment/train.py --img 256 --batch 128 --epochs 300 --data train_data/{data_name}/configs/fold_1.yaml --cfg yolov5/models/segment/{model} --device 5 --no-overlap --optimizer AdamW --patience 300